# Genome Informatics 2021 - Homework no. 2
## BAM file analysis
### Introduction

The second assignment in the Genome Informatics Course deals with the information extraction from an example BAM file. In order to gain the deeper understanding of how the data is being stored after the alignment process, students are required to perform simple tasks on the given data and present them in the form of a *Jupyter Notebook*. Data inspection and manipulation is performed with the help of *Pysam* module.

SAM stands for Sequence Alignment/Map format. It is a TAB-delimited text format consisting of a header
section, which is optional, and an alignment section. 
A BAM file (\*.bam) is the compressed binary version of a SAM file

Additionaly, every BAM file can have its corresponding index file BAI (\*.bai). BAI file acts like an external table of contents, and allows programs to jump directly to specific parts of the BAM file without reading through all of the sequences.

### Exercise

In order to complete the assignment, we will have to perform the next steps:
- Create an *AlignmentFile* object from a BAM file (*merged-tumor.bam*);
- Take the first read from the *AlignmentFile* object and check its fields;
- Check the flag column for some reads;
- Calculate:
  - Total number of reads;
  - Number of unmapped reads;
  - Number of reads with mapping quality 0;
  - Average mapping quality of the reads (with and without the filtering of zero-quality reads).

In the first step, we are creating an *AlignmentFile* object.

In [3]:
import pysam
bamfile = pysam.AlignmentFile("/sbgenomics/project-files/merged-tumor.bam", "rb")

The next step is the initialization of all the counters and sums we are going to need. We are also allocating the memory for the first read, in order to inspect it later.

In [4]:
first_read = pysam.AlignedSegment
num_of_reads = 0
num_of_unmapped_reads = 0
num_of_zero_q_reads = 0
total_average_quality = 0
filtered_average_quality = 0

The main processing is performed within a *for* loop. For every read in the BAM file, we are inspecting certain fields and incrementing our counters and sums accordingly.

In [5]:
for read in bamfile:
    # Extract the first read
    if num_of_reads == 0:
        first_read = read

    # Check for the unmapped reads
    if read.is_unmapped:
        num_of_unmapped_reads = num_of_unmapped_reads + 1

    # Check for the zero quality reads
    if read.mapping_quality == 0:
        num_of_zero_q_reads = num_of_zero_q_reads + 1
    else:
        filtered_average_quality = filtered_average_quality + read.mapping_quality

    # Total number of reads
    num_of_reads = num_of_reads + 1

bamfile.close()

Finaly, let's see the results:

In [6]:
print("Total number of reads in the file is " + str(num_of_reads))
print("Total number of unmapped reads in the file is " +  str(num_of_unmapped_reads))
print("Total number of zero-quality reads is " + str(num_of_zero_q_reads))
print("Average quality of all reads is " + str(filtered_average_quality/num_of_reads))
print("Average quailty of filtered reads is " + str(filtered_average_quality/(num_of_reads-num_of_zero_q_reads)))

Total number of reads in the file is 2921629
Total number of unmapped reads in the file is 17765
Total number of zero-quality reads is 126628
Average quality of all reads is 55.91379158681681
Average quailty of filtered reads is 58.446975510921106


Additionaly, we are going to get a closer look into what the content of the *AlignedSegment* object is.

In [7]:
print(first_read)

C0HVYACXX120402:7:1207:5722:57044	1187	20	9483248	27	76M	20	9483381	76	TTTTCAAACAGTATCTATGCCTGCCAAATGTGAACATATAAAAAAAAACCAGAATGTGCCATTCTGATTTAAACTG	array('B', [28, 28, 27, 29, 31, 30, 31, 31, 29, 31, 35, 30, 29, 31, 34, 30, 29, 23, 41, 32, 20, 30, 29, 34, 34, 29, 30, 31, 30, 30, 30, 33, 33, 26, 39, 12, 25, 19, 32, 30, 35, 28, 35, 33, 23, 33, 35, 36, 30, 38, 33, 41, 34, 35, 31, 33, 23, 30, 30, 36, 27, 32, 29, 34, 35, 41, 33, 31, 33, 29, 32, 32, 31, 31, 31, 34])	[('XA', 'GL000217.1,-110754,76M,1;'), ('BD', 'NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN'), ('MD', '76'), ('RG', '1'), ('BI', 'NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN'), ('NM', 0), ('MQ', 27), ('AS', 76), ('XS', 71)]


Let's take a closer look at the flag field.

In [9]:
first_read_flag = first_read.flag
print(first_read_flag)

1187


Let's define a function that will help us analyse the value of the flag field better.

In [8]:
def flag_field_info(flag):
    print("The value of the flag is: " + str(flag))
    if flag & 0x001:
        print("Template has multiple segments in sequencing.")
    if flag & 0x002:
        print("Each segment is properly aligned according to the aligner.")
    if flag & 0x004:
        print("The segment is unmapped.")
    if flag & 0x008:
        print("The next segment in the template is unmapped.")
    if flag & 0x010:
        print("SEQ is reverse complemented.")
    if flag & 0x020:
        print("SEQ of the next segment in the template is reverse complemented.")
    if flag & 0x040:
        print("The first segment in the template.")
    if flag & 0x080:
        print("The last segment in the template.")
    if flag & 0x100:
        print("Secondary alignment.")
    if flag & 0x200:
        print("Not passing filters, such as platform/vendor quality controls.")
    if flag & 0x400:
        print("PCR or optical duplicate.")
    if flag & 0x800:
        print("Supplementary alignment")

Finally, let's check the flag for our first read.

In [10]:
flag_field_info(first_read_flag)

The value of the flag is: 1187
Template has multiple segments in sequencing.
Each segment is properly aligned according to the aligner.
SEQ of the next segment in the template is reverse complemented.
The last segment in the template.
PCR or optical duplicate.
